Generate the predictions of a transferred network

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!cp -a /gdrive/MyDrive/DL_beat_detection .

In [ ]:
import os

In [ ]:
os.chdir('DL_beat_detection/data')

In [ ]:
#!tar xfz MIT_BIH_LongTerm.tar.gz 
#!tar xfz MIT_BIH_NormalSinus.tar.gz 
!tar xfz MIT_BIH_Arrhythmia.tar.gz 

In [ ]:
os.chdir('..')

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
import sys

from network import SilvaNet

from dataset import BeatDataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#EXPERIMENT SETTINGS
MODEL = SilvaNet
NAME_EXPERIMENT = 'transfer_SilvaNet_ComfTech_Movement'
DATASET = ['WCS_ComfTech_movement']
PARTITION= 'test'
N = 7443

DATAROOT = './data'


In [ ]:
#LOAD DATASETS
dataset = BeatDataset(f'{DATAROOT}/dataset',
                      [f'{x}/{PARTITION}' for x in DATASET],
                      N=N)

In [ ]:
state_dict = torch.load(f'{DATAROOT}/results/{NAME_EXPERIMENT}/weights.pth')

model = MODEL()
model.load_state_dict(state_dict)
model = model.to(device)

In [ ]:
model.eval()

results = []

with torch.no_grad():
    for sample in dataset:
        signal = sample["data"].unsqueeze(0).to(device)
        target = sample["target"]
        output = model(signal) #forward
        _, pred = torch.max(output,1)
        
        results_sample = {}
        results_sample['pred'] = pred.data.cpu().numpy()
        results_sample['target'] = int(target)
        
        results.append(pd.DataFrame(results_sample, index = [sample['sample']]))


In [ ]:
results = pd.concat(results)
results.to_csv(f'{DATAROOT}/predictions/{NAME_EXPERIMENT}___{PARTITION}.csv')

In [ ]:
f'{DATAROOT}/predictions/{NAME_EXPERIMENT}___{PARTITION}.csv'
